In [9]:
import pickle

Test = pickle.load(open(r"C:\Users\Admin\Documents\GitHub\DD-Net-Pytorch\data\JHMDB\GT_test_1.pkl", "rb"))
Train = pickle.load(open(r"C:\Users\Admin\Documents\GitHub\DD-Net-Pytorch\data\JHMDB\GT_train_1.pkl", "rb"))

In [26]:
Train['pose'][0].shape

(39, 15, 2)

In [28]:
Train['label'][0]

'clap'

In [19]:
uiprmd_train = pickle.load(open(r"C:\Users\Admin\Documents\GitHub\DD-Net-Pytorch\UIPRMD_pickle\EX_2\train_dataset.pkl", "rb"))
uiprmd_test = pickle.load(open(r"C:\Users\Admin\Documents\GitHub\DD-Net-Pytorch\UIPRMD_pickle\EX_2\test_dataset.pkl", "rb"))

In [30]:
uiprmd_train.keys()

dict_keys(['pose', 'labels'])

In [25]:
uiprmd_train['pose'].shape

torch.Size([88, 194, 39, 3])

In [24]:
uiprmd_train['pose'][0].shape


torch.Size([194, 39, 3])

In [31]:
uiprmd_train['labels'][0]

tensor([0.6286])

In [32]:
import numpy as np
lb = np.copy(uiprmd_train['labels'][0])

In [34]:
lb

array([0.62861], dtype=float32)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchdiffeq import odeint


class ODEFunc(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, dim // 2),
        )

    def forward(self, x):
        return self.net(x)


class ODEBlock(nn.Module):
    def __init__(self, odefunc, tol=1e-3):
        super().__init__()
        self.odefunc = odefunc
        self.tol = tol

    def forward(self, x):
        # x: (B, T, D)
        B, T, D = x.shape
        x = x.reshape(B * T, D)
        t = torch.tensor([0, 1]).float().to(x.device)
        out = odeint(self.odefunc, x, t, rtol=self.tol, atol=self.tol)
        out = out[1]
        out = out.view(B, T, D)
        return out

In [35]:
#!/usr/bin/env python
# coding:utf-8

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torchdiffeq import odeint


class c1D(nn.Module):
    # input (B,C,D) //batch,channels,dims
    # output = (B,C,filters)
    def __init__(self, input_channels, input_dims, filters, kernel):
        super(c1D, self).__init__()
        self.cut_last_element = (kernel % 2 == 0)
        self.padding = math.ceil((kernel - 1)/2)
        self.conv1 = nn.Conv1d(input_dims, filters,
                               kernel, bias=False, padding=self.padding)
        self.bn = nn.BatchNorm1d(num_features=input_channels)

    def forward(self, x):
        # x (B,D,C)
        x = x.permute(0, 2, 1)
        # output (B,filters,C)
        if self.cut_last_element:
            output = self.conv1(x)[:, :, :-1]
        else:
            output = self.conv1(x)
        # output = (B,C,filters)
        output = output.permute(0, 2, 1)
        output = self.bn(output)
        output = F.leaky_relu(output, 0.2, inplace=True)
        return output


class spatialDropout1D(nn.Module):
    def __init__(self, p):
        super(spatialDropout1D, self).__init__()
        self.dropout = nn.Dropout1d(p)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)
        return x


class ODEFunc(nn.Module):
    def __init__(self, channels, filters):
        super(ODEFunc, self).__init__()
        self.filters = filters
        # Use c1D to mimic convolutional dynamics
        self.conv = c1D(channels, filters, filters, 3) 
        self.norm = nn.LayerNorm(filters)

    def forward(self, t, x):
        # x: (B, frame_l, filters)
        x = self.conv(x)
        x = self.norm(x)
        return torch.relu(x)


class SlowODE(nn.Module):
    def __init__(self, frame_l, joint_n, joint_d, filters):
        super(SlowODE, self).__init__()
        self.frame_l = frame_l
        self.joint_n = joint_n
        self.joint_d = joint_d
        self.filters = filters

        # Initial transformation to match filters
        self.init_conv = nn.Sequential(
            c1D(frame_l, joint_n * joint_d, filters, 1),
            spatialDropout1D(0.1)
        )

        # ODE block
        self.ode_func = ODEFunc(frame_l, filters)
        self.ode_dropout = spatialDropout1D(0.1)

        # Pooling to reduce temporal dimension
        self.pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=2),
            spatialDropout1D(0.1)
        )

    def forward(self, x):
        # x: (B, frame_l, joint_n * joint_d)
        B, frame_l, _ = x.shape
        if frame_l != self.frame_l:
            raise ValueError(f"Input frame_l mismatch: expected {self.frame_l}, got {frame_l}")

        # Initial transformation
        x = self.init_conv(x)  # (B, frame_l, filters)
        
        # ODE integration
        t_span = torch.linspace(0, 1, 5, device=x.device)
        # x = odeint(self.ode_func, x, t_span, method='rk4')[-1]  # (B, frame_l, filters)
        x = odeint(self.ode_func, x, t_span, method='dopri5')[-1]
        x = self.ode_dropout(x)

        # Pooling
        x = x.permute(0, 2, 1)  # (B, filters, frame_l)
        x = self.pool(x)  # (B, filters, frame_l//2)
        x = x.permute(0, 2, 1)  # (B, frame_l//2, filters)
        
        return x


if __name__ == "__main__":
    # Example usage
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    frame_l, joint_n, joint_d, filters = 32, 15, 2, 64
    model = SlowODE(frame_l, joint_n, joint_d, filters).to(device)
    x = torch.randn(32, frame_l, joint_n * joint_d).to(device)  # Batch of 2
    output = model(x)
    print("Output shape:", output.shape)  # Expected: (2, 32, 64)
    print("Input shape:", x.shape)  # Expected: (2, 32, 64)


Output shape: torch.Size([32, 16, 64])
Input shape: torch.Size([32, 32, 30])


In [1]:
import torch
from torch import nn
from models.ulsam import ULSAM
# Giả sử input có 64 channels, kích thước 32x32
nin = 64
nout = 64  # thông thường bằng nin, vì ULSAM không thay đổi số channels
h, w = 32, 32
num_splits = 4

# Tạo mô-đun
ulsam = ULSAM(nin=nin, nout=nout, h=h, w=w, num_splits=num_splits).cuda()

# Tạo đầu vào giả (batch_size=8)
x = torch.randn(8, nin, h, w).cuda()

# Forward
out = ulsam(x)

print("Input shape: ", x.shape)
print("Output shape: ", out.shape)


Input shape:  torch.Size([8, 64, 32, 32])
Output shape:  torch.Size([8, 64, 32, 32])


In [ ]:
class simam_module(torch.nn.Module):
    def __init__(self, channels = None, e_lambda = 1e-4):
        super(simam_module, self).__init__()

        self.activaton = nn.Sigmoid()
        self.e_lambda = e_lambda

    def __repr__(self):
        s = self.__class__.__name__ + '('
        s += ('lambda=%f)' % self.e_lambda)
        return s

    @staticmethod
    def get_module_name():
        return "simam"

    def forward(self, x):

        b, c, h, w = x.size()
        
        n = w * h - 1

        x_minus_mu_square = (x - x.mean(dim=[2,3], keepdim=True)).pow(2)
        y = x_minus_mu_square / (4 * (x_minus_mu_square.sum(dim=[2,3], keepdim=True) / n + self.e_lambda)) + 0.5

        return x * self.activaton(y)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Định nghĩa simam_module
class simam_module(nn.Module):
    def __init__(self, e_lambda=1e-4):
        super(simam_module, self).__init__()
        self.activation = nn.Sigmoid()
        self.e_lambda = e_lambda

    def forward(self, x):
        b, c, h, w = x.size()
        n = w * h - 1
        x_minus_mu_square = (x - x.mean(dim=[2, 3], keepdim=True)).pow(2)
        y = x_minus_mu_square / (4 * (x_minus_mu_square.sum(dim=[2, 3], keepdim=True) / n + self.e_lambda)) + 0.5
        return x * self.activation(y)

class EleAttG_GRU_SimAM(nn.Module):
    def __init__(self, embedding_dim, n_hidden=128, n_classes=None):
        super(EleAttG_GRU_SimAM, self).__init__()

        assert n_classes is not None

        self.embedding_dim = embedding_dim
        self.n_hidden = n_hidden
        self.n_classes = n_classes

        self.attention = simam_module(e_lambda=1e-4)
        self.grucell = nn.GRUCell(self.embedding_dim, self.n_hidden)
        self.fc = nn.Sequential(
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, X):
        """
        X: [batch_size, frames, embedding_dim]
        """
        h = torch.zeros(X.shape[0], self.n_hidden).to(X.device)

        X = X.permute(0, 2, 1).unsqueeze(-1)  # [batch, embedding_dim, frames, 1]
        X = self.attention(X)  # [batch, embedding_dim, frames, 1]
        X = X.squeeze(-1).permute(0, 2, 1)  # [batch, frames, embedding_dim]

        for i in range(X.shape[1]):
            h = self.grucell(X[:, i, :], h)

        return self.fc(h)

In [3]:
if __name__ == "__main__":
    batch_size = 4
    seq_len = 10  # số lượng frames
    embedding_dim = 32
    n_hidden = 64
    n_classes = 5

    # Tạo dữ liệu giả
    X_fake = torch.randn(batch_size, seq_len, embedding_dim)

    # Khởi tạo model
    model = EleAttG_GRU_SimAM(embedding_dim=embedding_dim, n_hidden=n_hidden, n_classes=n_classes)

    # Chạy forward
    output = model(X_fake)

    # In output
    print("Output shape:", output.shape)  # Expect: [batch_size, n_classes]
    print("Output:", output)


Output shape: torch.Size([4, 5])
Output: tensor([[0.2065, 0.2216, 0.2010, 0.1862, 0.1848],
        [0.1990, 0.2119, 0.2008, 0.1953, 0.1930],
        [0.1985, 0.2211, 0.2068, 0.1849, 0.1887],
        [0.1930, 0.2227, 0.1976, 0.1919, 0.1948]], grad_fn=<SoftmaxBackward0>)
